In [1]:
import pymysql

# your username and password go here!
creds = {'user': 'fausto', 
         'pswd': 'dapuzzo'} 


connection = pymysql.connect(host="atriskwiki.cbprs6vpqcbz.us-west-1.rds.amazonaws.com",
                            user=creds['user'],
                            password=creds['pswd'],
                            port=3306,
                            db="atriskwiki",
                            charset="utf8")

cur = connection.cursor()

In [2]:
import pandas as pd
cl_epa = pd.read_csv("..\CategoryAndPageLinks\cl_epa_engineered.csv")
cl_nepa = pd.read_csv("..\CategoryAndPageLinks\cl_nepa_engineered.csv")

In [ ]:
import numpy as np
cl_nepa.shape

In [5]:
col_for_db = cl_nepa.columns
col_for_db[:3]

Index(['page_id', '1961_births', '1967_births'], dtype='object')

In [6]:
single = "{} INT(8) UNSIGNED NOT NULL"
createQ = list()
for i in range(len(col_for_db)):
    createQ.append(single.format(col_for_db[i]))    

In [7]:
processing = ", ".join(createQ)

In [50]:
processing[:100]

'page_id INT(8) UNSIGNED NOT NULL, 1961_births INT(8) UNSIGNED NOT NULL, 1967_births INT(8) UNSIGNED '

In [11]:
tail = "PRIMARY KEY (page_id)) DEFAULT CHARACTER SET utf8"
CL_create = ", ".join([processing,tail])
header = "CREATE TABLE IF NOT EXISTS categorylinks ("
CL_create_f = " ".join([header, CL_create])

In [46]:
cur.execute(CL_create_f)

0

In [13]:
import numpy as np
redundant = pd.merge(cl_epa, cl_nepa, how='inner', left_on="page_id", right_on="page_id")
redundantIDs = redundant['page_id'].tolist()
cl_nepa_filtered = cl_nepa.loc[~cl_nepa['page_id'].isin(redundantIDs)].copy()

In [19]:
print(len(redundantIDs))

73


In [20]:
master_links = pd.concat([cl_epa, cl_nepa_filtered])
master_links.shape

(83287, 91)

In [28]:
np.sum(master_links["1967_births"])

207

In [22]:
master_links = master_links.where((pd.notnull(master_links)), None) #converts NaNs to None type, required for mySQL

In [24]:
ma_for_db = master_links[col_for_db].to_dict(orient='records') #provides each row as a dict;
#keys are colnames, values are column values, the order of keys is retained from row to row

In [47]:
cur.execute("SET autocommit=0") #makes batch inserts SO MUCH EASIER

0

In [54]:
#generates the template query for insertions into db
insertQ = "INSERT IGNORE INTO pagelinks({}) VALUES ({})"
cols = ", ".join(col_for_db)
val_format = ",".join(['%s'] * len(list(ma_for_db[0].keys())))
formatQ = insertQ.format(cols, val_format)
formatQ

'INSERT IGNORE INTO pagelinks(page_id, 1961_births, 1967_births, 1971_births, 1977_births, 1992_births, 20th_century_american_actresses, 20th_century_american_male_actors, 20th_century_american_singers, 20th_century_american_writers, 21st_century_american_actresses, 21st_century_american_businesspeople, 21st_century_american_male_actors, 21st_century_american_singers, 21st_century_american_writers, _alumni, _authors_list, _california, _contestants, _films, _members, _multiple_names, _winners, african_american_basketball_players, african_american_male_rappers, all_accuracy_disputes, all_npov_disputes, american_animated_films, american_child_actresses, american_children, american_christians, american_female_models, american_female_pop_singers, american_film_actresses, american_film_producers, american_hip_hop_singers, american_male_child_actors, american_male_film_actors, american_male_singer_songwriters, american_male_television_actors, american_male_voice_actors, american_male_writers,

In [52]:
print(type(ma_for_db))

<class 'list'>


In [55]:
for record in ma_for_db:
    values = [str(x) for x in list(record.values())]
    cur.execute(formatQ,values)

connection.commit()

InternalError: (1054, "Unknown column '1961_births' in 'field list'")

In [44]:
cur.execute("SELECT * FROM pagelinks where page_id = 25 LIMIT 10")

1

In [45]:
rows = cur.fetchall() #returns tuple of tuples, preserving column order
easyPeasy = pd.DataFrame(list(rows), columns = ["page_id", "pl_count", "pl_is_epa", "ratio"])
easyPeasy

,page_id,pl_count,pl_is_epa,ratio
0,25,672,23,34
